In [2]:
import numpy as np
import pandas as pd
import redis

In [3]:
hostname='redis-17790.c281.us-east-1-2.ec2.cloud.redislabs.com'
portnumber=17790  
password='GLZP9gx3pWKk9NrVc00YjsRnhsOb3yEz'

In [4]:
r=redis.StrictRedis(host=hostname,port=portnumber,password=password)

In [5]:
r

Redis<ConnectionPool<Connection<host=redis-17790.c281.us-east-1-2.ec2.cloud.redislabs.com,port=17790,db=0>>>

In [6]:
r.ping()

True

In [7]:
#Load the data using numpy zip 
np_file=np.load('dataframe_students_teachers.npz',allow_pickle=True)
x_values=np_file['arr_0']
col_names=np_file['arr_1']

df=pd.DataFrame(x_values,columns=col_names)

In [8]:
np_file

NpzFile 'dataframe_students_teachers.npz' with keys: arr_0, arr_1

In [9]:
df

,Name,Role,Facial Features
0,Angelina Jolie,Student,"[0.1987483, 1.4748628, -0.09814876, -0.8804337..."
1,Barack Obama,Teacher,"[-0.22192378, 1.5761815, -0.5031222, 0.1360412..."
2,Chris Evans,Student,"[0.35674706, -0.41496292, 0.039872665, -0.1544..."
3,Morgan Freeman,Teacher,"[0.7627855, 0.8906721, 0.46884507, 0.25902992,..."
4,Scarlett Johansson,Student,"[-0.13563304, -0.28201127, -0.4271903, -0.2822..."


In [10]:
df['name_role']=df['Name']+'@'+df['Role'] 
df

,Name,Role,Facial Features,name_role
0,Angelina Jolie,Student,"[0.1987483, 1.4748628, -0.09814876, -0.8804337...",Angelina Jolie@Student
1,Barack Obama,Teacher,"[-0.22192378, 1.5761815, -0.5031222, 0.1360412...",Barack Obama@Teacher
2,Chris Evans,Student,"[0.35674706, -0.41496292, 0.039872665, -0.1544...",Chris Evans@Student
3,Morgan Freeman,Teacher,"[0.7627855, 0.8906721, 0.46884507, 0.25902992,...",Morgan Freeman@Teacher
4,Scarlett Johansson,Student,"[-0.13563304, -0.28201127, -0.4271903, -0.2822...",Scarlett Johansson@Student


In [11]:
records=df[['name_role','Facial Features']].to_dict(orient='records')

In [12]:
records

[{'name_role': 'Angelina Jolie@Student',
  'Facial Features': array([ 1.98748305e-01,  1.47486281e+00, -9.81487632e-02, -8.80433679e-01,
         -4.95913208e-01,  6.36685610e-01,  7.20437050e-01,  4.32217658e-01,
         -1.01355433e+00, -2.19099611e-01, -1.25162935e+00,  6.21991139e-03,
         -1.08987772e+00, -9.10613060e-01,  9.80052352e-01, -8.08841527e-01,
         -3.21958333e-01, -3.24594975e-01, -1.63132936e-01, -2.94654667e-01,
          6.83650613e-01,  3.98857713e-01, -3.36551249e-01, -3.36880744e-01,
         -1.28378794e-01,  1.73572212e-01,  5.91753364e-01, -6.08514190e-01,
          1.05123878e+00,  7.50698745e-01,  5.75472936e-02, -4.44831669e-01,
          7.35051513e-01, -1.39626372e+00,  5.44246078e-01,  1.25618792e+00,
          1.07382022e-01, -3.43393534e-01, -1.06192574e-01,  1.95723116e+00,
          6.32386267e-01,  5.08046746e-01,  5.25688350e-01, -2.17110906e-02,
         -8.33714366e-01, -5.26804216e-02, -2.05193087e-01,  8.91342640e-01,
         -4.6958

In [12]:
len(records)

5

### Save data in Redis

In [33]:
for record in records:
    name_role=record['name_role']
    vector=record['Facial Features']
    
    
    vector_bytes=vector.tobytes()
    
    
    #save data in redis cloud
    r.hset(name='academy:register',key=name_role,value=vector_bytes)

### Registration Form

In [13]:
import cv2 
from insightface.app import FaceAnalysis

In [14]:
faceapp=FaceAnalysis(name='buffalo_l',root='insightface_models',providers=['CPUExecutionProvider']) 
faceapp.prepare(ctx_id=0,det_size=(640,640))

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_models\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_models\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_models\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_models\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_models\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [26]:
person_name=input("Enter your name: ")
trails=3
for i in range(trails):
    role=input("""
    Please Choose
    1. Student 
    2. Teacher
    
    Enter number either 1 or 2 
    """) 
    
    if role in('1','2'):
        if role=='1':
            role='Student'
        else:
            role='Teacher'
        break
        
    else:
        print('Invalid Entry!! Please try again')
        if i==3:
            print("Maximum entries exceeded") 
            
key=person_name+'@'+role 
print('Your name= ',person_name)
print('Your role= ',role)
print('Key=',key)
    

Enter your name: Santosh

    Please Choose
    1. Student 
    2. Teacher
    
    Enter number either 1 or 2 
    1
Your name=  Santosh
Your role=  Student
Key= Santosh@Student


In [29]:
person_name,role

('Santosh', 'Student')

In [30]:
cap=cv2.VideoCapture(0)
face_embeddings=[]
sample=0

while True:
    ret,frame=cap.read()
    if ret==False:
        print("Unable to read camera")
        break 
        
        
    #get results from insightface model
    results=faceapp.get(frame,max_num=1)
    for res in results:
        sample+=1
        x1,y1,x2,y2=res['bbox'].astype(int)
        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),1)
        
        
        embeddings=res['embedding']
        face_embeddings.append(embeddings)
    if sample>=200:
        break
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

In [24]:
frame

array([[[168, 180, 175],
        [169, 179, 176],
        [171, 178, 178],
        ...,
        [ 91,  94, 104],
        [ 81,  88,  98],
        [ 78,  87,  97]],

       [[170, 182, 177],
        [169, 180, 177],
        [171, 179, 178],
        ...,
        [ 86,  90,  99],
        [ 78,  87,  94],
        [ 77,  88,  95]],

       [[170, 183, 178],
        [170, 181, 178],
        [170, 180, 178],
        ...,
        [ 83,  90,  96],
        [ 80,  91,  95],
        [ 79,  92,  94]],

       ...,

       [[231, 231, 225],
        [232, 232, 226],
        [232, 232, 226],
        ...,
        [155,  91, 103],
        [153,  89, 102],
        [153,  89, 102]],

       [[231, 228, 226],
        [232, 229, 227],
        [230, 229, 226],
        ...,
        [154,  91, 103],
        [152,  89, 102],
        [152,  89, 102]],

       [[234, 230, 229],
        [232, 229, 228],
        [230, 228, 228],
        ...,
        [152,  90, 102],
        [150,  88, 100],
        [151,  89, 101]]

In [31]:
len(face_embeddings)

200

In [32]:
results

[{'bbox': array([278.61926, 116.25591, 459.34293, 361.9485 ], dtype=float32),
  'kps': array([[339.5592 , 209.37027],
         [424.5782 , 205.42058],
         [395.55914, 246.37054],
         [356.1296 , 302.3416 ],
         [424.0067 , 297.57263]], dtype=float32),
  'det_score': 0.83484685,
  'landmark_3d_68': array([[ 2.6426477e+02,  2.1702478e+02,  1.2444365e+02],
         [ 2.6995178e+02,  2.4414032e+02,  1.2131965e+02],
         [ 2.7658566e+02,  2.7172321e+02,  1.1602528e+02],
         [ 2.8489215e+02,  2.9759372e+02,  1.0633928e+02],
         [ 2.9853635e+02,  3.2021320e+02,  8.5650200e+01],
         [ 3.1582336e+02,  3.3644452e+02,  6.8290337e+01],
         [ 3.3526797e+02,  3.4825272e+02,  5.4256638e+01],
         [ 3.5854306e+02,  3.5627322e+02,  3.6578587e+01],
         [ 3.9142978e+02,  3.6176562e+02,  3.3454472e+01],
         [ 4.1710632e+02,  3.5335132e+02,  5.3049572e+01],
         [ 4.3139053e+02,  3.4128268e+02,  7.9563957e+01],
         [ 4.4251376e+02,  3.2428796e+0

In [36]:
x_mean

array([-0.01668193,  1.0066701 , -1.2527759 ,  0.37996042, -0.39353833,
       -0.42448866,  1.2077065 ,  0.11311842,  0.00993032,  0.64467   ,
       -0.92561793,  1.3677155 , -0.42468426,  0.85599875,  1.3358175 ,
        1.1542044 ,  0.31978473,  0.63567233, -0.45101756, -0.37106934,
        0.24812703, -0.6909161 , -1.2465416 ,  0.21197575,  0.06600843,
        0.562579  , -0.2783884 , -0.24829462,  0.22341195, -2.0303316 ,
        0.70922315, -0.70164657,  0.86662   ,  0.25390145,  0.40356514,
       -0.35527107, -0.11400479,  0.77222586,  0.24257968, -0.6915436 ,
        0.74669045, -1.2211953 ,  0.1359469 , -0.64361817, -0.0958692 ,
        1.7402912 ,  0.38938525,  0.461101  , -0.42793012, -0.82116586,
       -0.74203515, -0.5419121 ,  1.9515276 , -1.2271198 ,  0.8207441 ,
       -0.3049265 ,  0.19609357,  0.14791279, -0.31524125,  0.04874305,
        1.6773276 , -0.8340329 , -0.6739621 , -0.10895378,  0.8487611 ,
        0.10051299,  0.2426809 ,  1.485137  , -0.04894984, -1.38

In [37]:
x_mean=np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [38]:
x_mean_bytes=x_mean.tobytes()

In [40]:
r.hset(name='academy:register',key=key,value=x_mean_bytes)

1